In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

# 数据处理

In [ ]:
#载入数据
df = pd.read_excel(r"K:\zqh\data_B.xlsx")

In [ ]:
#制作X集合
X = df.drop(df.columns[3],axis=1)

#给col重新命名
X.columns=["date","time","XCO","XNMHC","XNOX","XNO2","XO3","XT","XAH"]

# # 时间改成整数
# for i in X.index:
#     X.time[i]=X.time[i].hour
# X.time = X.time.astype("int")

#时间和日期不要了
X = X.drop(["date","time"],axis=1)

#查看是否有异常值
for i in X.columns:
    X[i].plot()

In [ ]:
X

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(WP,WP*0.7),dpi=100)
sns.lineplot(data=X.drop(["XT","XAH"],axis=1),dashes=False,ax=ax)


In [ ]:
#制作target dataset，查看是否有异常值
y = pd.DataFrame(df.loc[:,df.columns[3]])
y.plot()

In [ ]:
# 39index是异常值，用38号的值修复一下
y = pd.DataFrame(df.loc[:,df.columns[3]])
#改一下col name
y.columns=["y"]
y.loc[39] = y.loc[38]

In [ ]:
#制作窗口为4的时间序列数组
Xn=X.copy()
yn=y.copy()
#只制作XNMHC的序列数组
tmp = pd.DataFrame(X.XNMHC)
#宽度是4
TW=4

for i in range(TW):
    #用index的次序覆盖，然后重编号
    Xn = Xn.iloc[1:].reset_index(drop=True)
    #加入，并且命名好新维度
    Xn["XNMHC-"+str(i+1)] = tmp[:-(i+1)].values

#更新对应的target
yn = yn.iloc[i+1:].reset_index(drop=True)

In [ ]:
#train dataset和test dataset的制作
def func0(Xn,yn):
    #中间隔24个样本（1天）
    Xn1,yn1 = Xn.iloc[:-24*2],yn.iloc[:-24*2]
    #下面是test dataset，其实也是按照index order来的
    Xn2,yn2 = Xn.iloc[-24:],yn.iloc[-24:]
    return Xn1,yn1,Xn2,yn2

数据池

In [ ]:
#数据池子的构造

data={}
# d1，单NMHC响应
Xa=pd.DataFrame(Xn.XNMHC)#only one
Xa1,ya1,Xa2,ya2=func0(Xa,yn)
data["d1"]=[Xa1,ya1,Xa2,ya2]


# d2，全部仪器响应
Xc=Xn
Xc1,yc1,Xc2,yc2=func0(Xc,yn)
data["d2"]=[Xc1.drop(["XNMHC-1","XNMHC-2","XNMHC-3","XNMHC-4"],axis=1),yc1,Xc2.drop(["XNMHC-1","XNMHC-2","XNMHC-3","XNMHC-4"],axis=1),yc2]

#d3，多通道序列
data["d3"]=[Xc1,yc1,Xc2,yc2]

In [ ]:
#标准化
#from sklearn.preprocessing import StandardScaler
#for i in data.keys():
#    data[i]
#    ssx=StandardScaler()
#    ssy=StandardScaler()
    
#    ssx.fit(data[i][0])
#    ssy.fit(data[i][1])
    
#    data[i][0]=pd.DataFrame(ssx.transform(data[i][0]),columns=data[i][0].columns)
#    data[i][1]=pd.DataFrame(ssy.transform(data[i][1]),columns=data[i][1].columns)
#    data[i][2]=pd.DataFrame(ssx.transform(data[i][2]),columns=data[i][2].columns)
#    data[i][3]=pd.DataFrame(ssy.transform(data[i][3]),columns=data[i][3].columns)
        
#    data[i].append([ssx,ssy])
    

# 模型构造

模型池

In [ ]:
models = {}
from sklearn.svm import SVR,LinearSVR
from sklearn.linear_model import LinearRegression,ElasticNetCV

# # 0.1 better-rbf-svr
models["better-svr"]=615

# # 1.1 linear-svr
models["lsvr"]=LinearSVR(random_state=615,max_iter=1000)

# # 1.2 rbf-svr
models["rbfsvr"]=SVR(C=100,max_iter=1000)#狂野调参，要不太废物了啊

# # 2.1 普通偏最小二乘回归
models["oplsr"]=LinearRegression()

# # 2.2 cv3的弹性网回归
models["en3"]=ElasticNetCV(cv=3, random_state=615)

better svm

In [ ]:
def better_rbfsvm(X,y):
    from hyperactive import Hyperactive
    from hyperactive.optimizers import SimulatedAnnealingOptimizer
    from sklearn.model_selection import cross_val_score
    
    y=y.values.ravel()#让warning闭嘴
    
    ##1.主干部分
    def model(opt):
        svr = SVR(C=opt["C"],#C是控制正则惩罚的第一个
                  gamma=opt["gamma"],#这个是控制rbf核的
                  epsilon=opt["epsilon"],#epsilon是控制正则惩罚的另一个
                  cache_size=5000,
                  max_iter=1000)#limit the max iteration von svm 
        svr.fit(X, y)
        scores = cross_val_score(svr, X, y, cv=5)#cv5的交叉检验，这里默认的metrics是R2，我们就用它了
        #否则scoring="neg_mean_absolute_error"or"r2",都不用加负号，因为这个是找最大的（mae是负的）
        return scores.mean()
    
    ##2.search_space部分
    ###碎碎念一下，至少让初始化的地方，是默认值
    ###svm的话，可以先狂野的搞一下C，10，100，1000这样的
    
    search_space={}
    C=100#狂野调试
    gamma=1/(X.shape[1]*X.values.var())#"scale"模式下的取值
    epsilon=0.1#docs里面的默认值


    low  = list(np.linspace(start=1,stop=C,num=1000,dtype="int"))#C：100~1000，均匀取100个
    high = list(np.linspace(start=C,stop=C*10,num=9000,dtype="int"))#C：1000~10000，均匀取100个
    tmp=list(set(low+high))
    #tmp=list(set(list(np.linspace(start=100,stop=3000,num=3000,dtype="int"))))#C：1000~10000，均匀取100个
    tmp.sort()
    search_space["C"] = tmp#排重、衔接起来

    low  = list(np.linspace(start=gamma/10,stop=gamma,num=5000))#小一个数量级，然后到自身，均匀取5k个
    high = list(np.linspace(start=gamma,stop=gamma*10,num=5000))#自身到大一个数量级，均匀取5k个
    tmp=list(set(low+high))
    tmp.sort()
    search_space["gamma"] = tmp#排重、衔接

    low  = list(np.linspace(start=epsilon/10,stop=epsilon,num=5000))#同上
    high = list(np.linspace(start=epsilon,stop=epsilon*10,endpoint=False,num=5000))
    tmp=list(set(low+high))
    tmp.sort()
    search_space["epsilon"] = tmp

    ##3.初始空间
    kkp = {"C": C, "gamma": gamma, "epsilon":epsilon}#初始化数值的选择，也是个dict
    
    ##4.优化器部分
    optimizer = SimulatedAnnealingOptimizer(
        epsilon=0.5,
        distribution="normal",
        n_neighbours=5,
        rand_rest_p=0,#随机重启率
        annealing_rate=0.90,#退火率
        start_temp=1,)
    
    ##干活了
    hyper = Hyperactive(verbosity = ["progress_bar", "print_results", "print_times"],distribution ="joblib")
    hyper.add_search(model, #def的主干部分
                     search_space, #搜索空间
                     optimizer=optimizer,#优化器部分
                     initialize={ "warm_start": [kkp]}, #初始空间
                     n_jobs=1,
                     n_iter=10000)#多少次
    #开工
    hyper.run()#max_time = 600)#你可以限制时间
        
    return hyper.best_para(model),hyper.results(model)

# flow space

In [ ]:
last_results={}
hyper_results=[]
for i in data.keys():
    for j in models.keys():
        if j[0:6]=="better":#单独为better_svm设计的
            
            paras,resl = better_rbfsvm(data[i][0],data[i][1])
            #paras的最好的parameters
            #resl是迭代过程的结果
            
            C=paras["C"]
            gamma=paras["gamma"]
            epsilon=paras["epsilon"]
            
            hyper_results.append([paras,resl])
            model = SVR(C=C,gamma=gamma,epsilon=epsilon)
            model.fit(data[i][0],data[i][1])
            
            yhat1=model.predict(data[i][0])
            yhat2=model.predict(data[i][2])
            
        else:#其他陪跑选手
            #拟合部分
            model = models[j]
            model.fit(data[i][0],data[i][1])
            
            #保存予测值
            yhat1=model.predict(data[i][0])
            yhat2=model.predict(data[i][2])
        
        #output部分
        last_results[str(j)+"_"+str(i)]={"model":model,"yhat1":yhat1,"yhat2":yhat2,"data":data[i]}
        #last_results的部分：
        ## "model":拟合好的模型
        ## "yhat1":训练集的结果
        ## "yhat2":测试集的结果
        ## "data"：数据池，0，1=训练，2，3=测试 
        

# 小报告

In [ ]:
#定义报告函数
def func_report(y_true,y_pred):
    
    from sklearn import metrics
    reports={"r2" : metrics.r2_score(y_true,y_pred),
             "mae" : metrics.mean_absolute_error(y_true,y_pred),
             "mape" : 1-metrics.mean_absolute_percentage_error(y_true,y_pred),#折算的
             "mxr" : metrics.max_error(y_true,y_pred),#max error
             "medae" : metrics.median_absolute_error(y_true,y_pred)}#medium ae中位数误差
    
    return reports    

In [ ]:
metrics_test={}
metrics_train={}

for i in last_results.keys():
    #print(i)    
    metrics_train[i]=func_report(last_results[i]["data"][1],last_results[i]["yhat1"])
    metrics_test[i]=func_report(last_results[i]["data"][3],last_results[i]["yhat2"]) 

    
train_reports=pd.DataFrame(columns=metrics_train[i].keys(),index=metrics_train.keys())#train的做成pd.Dataframe
for i in train_reports.index:
    train_reports.loc[i]=list(metrics_train[i].values())
    
test_reports=pd.DataFrame(columns=metrics_test[i].keys(),index=metrics_test.keys())#test的做成pd.Dataframe
for i in test_reports.index:    
    test_reports.loc[i]=list(metrics_test[i].values())



test_reports.to_excel(r"K:\修理亮\kkps.xlsx")

In [ ]:
test_reports

# nice!

import joblib
joblib.dump(train_reports,r"K:\修理亮\data\train_reports.zqh")
joblib.dump(test_reports,r"K:\修理亮\data\test_reports.zqh")
joblib.dump(hyper_results,r"K:\修理亮\data\hyper_results.zqh")
joblib.dump(last_results,r"K:\修理亮\data\last_results.zqh")
joblib.dump(data,r"K:\修理亮\data\data.zqh")

# recovery

In [ ]:
import joblib
train_reports=joblib.load(r"K:\zqh\data\train_reports.zqh")
test_reports=joblib.load(r"K:\zqh\data\test_reports.zqh")
hyper_results=joblib.load(r"K:\zqh\data\hyper_results.zqh")
last_results=joblib.load(r"K:\zqh\data\last_results.zqh")
data=joblib.load(r"K:\zqh\data\data.zqh")

In [ ]:
test_reports

# 画图

In [ ]:
plt.rcParams["font.family"]="Roboto"
plt.rcParams["font.size"]=7.5
plt.rcParams['figure.figsize'] = 3,3
WP=3
sns.set()
sns.set_style("ticks")

font = {'family' : 'Roboto',
        'size'   : 7.5}

matplotlib.rc('font', **font)

## 效果图

In [ ]:
hats2=pd.DataFrame()
hats2["ytrue"] = data["d1"][3]["y"].values
hats2["oplsr_d1"]=last_results["oplsr_d1"]["yhat2"]
hats2["oplsr_d2"]=last_results["oplsr_d2"]["yhat2"]
hats2["oplsr_d3"]=last_results["oplsr_d3"]["yhat2"]
hats2["better-svr_d3"]=last_results["better-svr_d3"]["yhat2"]

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(WP*2.5,WP),dpi=100)
sns.lineplot(data=hats2,markers=False,legend=True,sizes=0.01,ax=ax,dashes=False)

plt.title("pred vs true")
sns.despine()

plt.savefig(r"K:\zqh\figs\fig8.png")

# kkp

In [ ]:
xcs=[]
for i in test_reports.index.values:
    xcs.append(i[-2:])
    
sbx=test_reports.index.values
sbr2=test_reports.r2.values
sbmape=test_reports.mape.values
sbmae=test_reports.mae.values

sbx2=[]
for i in sbx:
    sbx2.append(i[:-3])
    
snsb=pd.DataFrame([sbx2,sbr2,sbmape,sbmae,xcs]).T.rename({0:"model",1:"r2",2:"mape",3:"mae",4:"data_style"},axis=1)

## R2

In [ ]:
fig, ax = plt.subplots(dpi=100)

ax = sns.barplot(x="model",y="r2",hue="data_style",data=snsb,ax=ax)

ax.set_xlabel(None)
ax.set_ylabel(None)

plt.title("R2s")
plt.legend(loc=4)
plt.ylim([0.4,1.0])

sns.despine()
#plt.savefig(r"K:\zqh\figs\fig6.png")

## MAPE

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax=sns.barplot(x="model",y="mape",hue="data_style",data=snsb,ax=ax)

ax.set_xlabel(None)
ax.set_ylabel(None)

plt.title("MAPEs")
plt.legend(loc=4)
plt.ylim([0.4,1.0])

sns.despine()

plt.savefig(r"K:\zqh\figs\fig6.png")

## MAE

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax = sns.barplot(x="model",y="mae",hue="data_style",data=snsb,ax=ax,palette=("brg_r"))


ax.set_xlabel(None)
ax.set_ylabel(None)

plt.title("MAEs")
plt.legend(loc=1)
plt.ylim([0,100])

sns.despine()

## 雷达图

In [ ]:
test_reports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from math import pi
 
# Set data
deltas=pd.DataFrame()
deltas["oplsr_d1"]=test_reports[["mae","mxr","medae"]].iloc[3]
deltas["oplsr_d2"]=test_reports[["mae","mxr","medae"]].iloc[8]
deltas["oplsr_d3"]=test_reports[["mae","mxr","medae"]].iloc[13]
deltas["better-svr_d2"]=test_reports[["mae","mxr","medae"]].iloc[5]
deltas["better-svr_d3"]=test_reports[["mae","mxr","medae"]].iloc[10]

deltas["group"]=deltas.index.values
deltas=deltas.reset_index(drop=True)
deltas = deltas.T.reset_index().iloc[:-1].rename({"index":"group",0:"mae",1:"mxr",2:"med"},axis=1)
df = deltas
# ------- PART 1: Create background
 
# number of variable
categories=list(df)[1:]
N = len(categories)
 
# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]
 
# Initialise the spider plot
ax = plt.subplot(111, polar=True)
 
# If you want the first axis to be on top:
ax.set_theta_offset(pi / 2)
ax.set_theta_direction(-1)
 
# Draw one axe per variable + add labels
plt.xticks(angles[1:], categories)
 
# Draw ylabels
#ax.set_rlabel_position(0)
#plt.yticks([10,20,30], ["10","20","30"], color="grey", size=7)
#plt.ylim(0,40)
 

# ------- PART 2: Add plots
 
# Plot each individual = each line of the data
# I don't make a loop, because plotting more than 3 groups makes the chart unreadable


for i in range(len(df)):
    values=df.loc[i].drop('group').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=1, linestyle='solid', label=df.group.iloc[i],alpha=1)
    ax.fill(angles, values, 'b', alpha=0)

# Add legend
plt.legend(bbox_to_anchor=(0.36, 1.0))
plt.savefig(r"K:\zqh\figs\fig7.png")
# Show the graph
plt.show()



## 组图1：超参数3d

In [ ]:
import re, seaborn as sns
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap

# generate data
#n = 200
px = hyper_results[-1][-1].C.values
py = hyper_results[-1][-1].gamma.values
pz = hyper_results[-1][-1].epsilon.values
pc = hyper_results[-1][-1].score.values

pc2 = pd.DataFrame(pc).rename({0:"pc"},axis=1).sort_values("pc",ascending=False)
pc2["od"]=list(np.linspace(1,1/len(pc2),len(pc2)))
pc2=pc2.sort_index()
pc2=pc2.drop("pc",axis=1)

# axes instance
fig = plt.figure(figsize=(WP,WP),dpi=100)
ax = Axes3D(fig)
fig.add_axes(ax)

# get colormap from seaborn
cmap = ListedColormap(sns.color_palette("Spectral_r",100).as_hex())
#cmap = ListedColormap(sns.color_palette("Reds",100).as_hex())

# plot
sc = ax.scatter(px, py, pz, s=1, c=pc2, marker='o', cmap=cmap, alpha=1)
ax.set_xlabel('C')
ax.set_ylabel('gamma(           )')
ax.set_zlabel('epsilon')


fig.colorbar(sc,ax=ax,fraction=0.04, pad=0.2)
#plt.savefig("scatter_hue", bbox_inches='tight')

plt.savefig(r"K:\zqh\figs\fig5.png")
plt.savefig(r"K:\zqh\figs\fig5.svg", format="svg")

In [ ]:
fig,ax=plt.subplots(figsize=(WP,WP),dpi=100)
ax = sns.scatterplot(x=px,y=py,c=pc2.od.values,cmap=cmap,ax=ax,s=pc2*20)
ax.set_xlabel("C")
ax.set_ylabel("gamma")

In [ ]:
fig,ax=plt.subplots(figsize=(WP,WP),dpi=100)
ax = sns.scatterplot(x=py,y=pz,c=pc2.od.values,cmap=cmap,ax=ax,s=pc2*20)
ax.set_xlabel("gamma")
ax.set_ylabel("epsilon")

In [ ]:
fig,ax=plt.subplots(figsize=(WP,WP),dpi=100)
ax = sns.scatterplot(x=px,y=pz,c=pc2.od.values,cmap=cmap,ax=ax,s=pc2*20)
ax.set_xlabel("C")
ax.set_ylabel("epsilon")
nmb = matplotlib.cm.ScalarMappable(cmap=cmap)
fig.colorbar(nmb,ax=ax[2])

fig, ax = plt.subplots(nrows=1,ncols=3,figsize=(WP,WP),dpi=100)
nmb = matplotlib.cm.ScalarMappable(cmap=cmap)

sns.scatterplot(x=px,y=py,c=pc2.od.values,cmap=cmap,ax=ax[0],s=pc2*20)
#fig.colorbar(nmb,ax=ax[0])
sns.scatterplot(x=py,y=pz,c=pc2.od.values,cmap=cmap,ax=ax[1],s=pc2*20)
#fig.colorbar(nmb,ax=ax[1])
sns.scatterplot(x=px,y=pz,c=pc2.od.values,cmap=cmap,ax=ax[2],s=pc2*20)
#fig.colorbar(nmb,ax=ax[2])
plt.subplots_adjust()
#sns.despine()

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(WP,WP),dpi=100)
sns.scatterplot(data=hyper_results[2][1].score,s=(hyper_results[2][1].score+1)*2)
ax.set_xlabel("iteration")
ax.set_ylabel("score")

plt.xlim([0,5000])
plt.ylim([0.8,1.0])

In [ ]:
df0 = pd.read_excel(r"K:\zqh\data_A.xlsx").iloc[0:182]
df0 = df0.drop(['Date', 'Time',  'PT08.S1(CO)', 'PT08.S2(NMHC)','PT08.S3(NOx)', 'PT08.S4(NO2)','PT08.S5(O3)', 'T', 'AH','RH'],axis=1)
pms=df0.corr(method="pearson")
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(WP*1.5,WP),dpi=100)
sns.heatmap(pms,annot=True,ax=ax,cmap=cmap)
ax.set_xticklabels(ax.get_xticklabels(),rotation = -30)
pms

plt.savefig(r"K:\zqh\figs\fig4.png")
plt.savefig(r"K:\zqh\figs\fig4.svg", format="svg")

plt.show()